#### Import needed

In [59]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import pickle as pkl
from datasets import Dataset

#### Getting all the links of the pages

In [46]:
last_page = 4150
link_list = []
for i in range(1, last_page+1):
    link_list.append("https://stats.stackexchange.com/questions?tab=votes&page={}".format(i))

#### Getting all the links of the questions

In [ ]:
questions = []
start = 0
end = 500
for link in tqdm(link_list[start:end]):
    page = requests.get(link)
    if page.status_code == 200:
        pageParsed = BeautifulSoup(page.content, 'html.parser')
        try:
            all_page = pageParsed.find_all('div', {'class':'s-post-summary--content'})
            for question in all_page:
                question_link = question.find('h3', class_='s-post-summary--content-title').find('a')['href']
                questions.append('https://stats.stackexchange.com' + question_link)
        except:
            print('Failed')

#### Save the links list of the questions

In [63]:
with open('questions_links.pkl', 'wb') as file:
    pkl.dump(questions, file)

#### Open the list

In [67]:
with open('questions_links.pkl', 'rb') as file:
    questions_links = pkl.load(file)

#### Create a dictionary that contains:

- index

- question

- answer

In [126]:
df_dict = {}
idx = 1
for question_link in questions_links[0:5]:
    page = requests.get(question_link)
    answers_list = []
    if page.status_code == 200:
        pageParsed = BeautifulSoup(page.content, 'html.parser')
        try:
            texts = pageParsed.find_all('div', {'class':'s-prose js-post-body'})
            question = texts[0].text
            if question.startswith('\n'):
                question = question[1:]
            for answer in texts[1:]:
                df_dict[idx] = {'question': question,
                                'answer': answer.text}
                idx += 1
        except:
            print('Failed')

#### Create a dataframe and then a transformers Dataset from the dictionary

In [127]:
rows = [{'idx': key, 'question': value['question'], 'answer': value['answer']} for key, value in df_dict.items()]
df = pd.DataFrame(rows)
display(df.head())
dataset = Dataset.from_pandas(df)

,idx,question,answer
0,1,In today's pattern recognition class my profes...,\nImagine a big family dinner where everybody ...
1,2,In today's pattern recognition class my profes...,"\nThe manuscript ""A tutorial on Principal Comp..."
2,3,In today's pattern recognition class my profes...,\nLet's do (2) first. PCA fits an ellipsoid t...
3,4,In today's pattern recognition class my profes...,"\nHmm, here goes for a completely non-mathemat..."
4,5,In today's pattern recognition class my profes...,"\nI'd answer in ""layman's terms"" by saying tha..."


#### Save the dataset

In [ ]:
with open('dataset.pkl', 'wb') as file:
    pkl.dump(dataset, file)